# Filtro de extremos: Sobrecompra y Sobreventa + Posición Bull/Bear (RSI 50)

Funcionalidad que utiliza el indicador técnico RSI para determinar el momentum de los activos seleccionados: Sobrecompra (>= 70), medio (menor a 70 y mayor a 30), o bien Sobreventa (<= 30). Adicionalmente se informa la zona de tendencia del activo (muchas veces utilizada como filtro para sistemas de cruces de medias móviles). Si RSI >= 50: Zona Bull, sino, Zona Bear. Devuelve tablas, tablas estiladas e incluso la info en un diccionario clasificando activos por nivel de SC, SV y zona de tendencia.

# INDICE DE FUNCIONES:
- filterSCSV(activos, timeframe, report = False):
    Esta función utiliza el indicador técnico RSI para determinar el momentum de los activos seleccionados: Sobrecompra (>= 70), medio 
    (menor a 70 y mayor a 30), o bien Sobreventa (<= 30). Adicionalmente se informa la zona de tendencia del activo (muchas veces utilizada 
    como filtro para sistemas de cruces de medias móviles). Si RSI >= 50: Zona Bull, sino, Zona Bear. Devuelve tablas, tablas estiladas e 
    incluso la info en un diccionario clasificando activos por nivel de SC, SV y zona de tendencia.

In [5]:
# FUNCIONES NECESARIAS PARA OBTENER DATA FINANCIERA:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

def getDataYf(ticker, tipo, interval, data_from = None, data_to = None, period = None):
    """
    Es una función para descargar market data de Yahoo Finance con la librería yfinance.
    
    ## Inputs:
        >ticker: el nombre del ticker.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    if tipo == "no end":
        data = yf.download(ticker, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(ticker, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(ticker, interval = interval, period = period, progress = False, auto_adjust = True)
    return data



def getDataYfMulti(activos, tipo, interval, data_from = None, data_to = None, period = None, swap = True):
    """
    Función para hacer batch requests (varios tickers a la vez), que será la fx que más voy a utilizar para market data.
    
    ## Inputs:
        >tickers: es una lista con los tickers de los cuales se va a obtener market data.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        >swap : si es True, divide el df en tickers y cada uno tiene su OHLC. Si es False, tenemos cada columna OHLC y dentro todos los tickers.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    lideres_arg = ["ALUA.BA", "BBAR.BA", "BMA.BA", "BYMA.BA", "CEPU.BA", "COME.BA", "CRES.BA", "CVH.BA", "EDN.BA", 
                   "GGAL.BA", "LOMA.BA", "MIRG.BA", "PAMP.BA", "SUPV.BA", "TECO2.BA", "TGNO4.BA", "TGSU2.BA", "TRAN.BA", 
                   "TXAR.BA", "VALO.BA", "YPFD.BA"]

    general_arg = ["AGRO.BA", "AUSO.BA", "BHIP.BA", "BOLT.BA", "BPAT.BA", "CADO.BA", "CAPX.BA", "CARC.BA", "CECO2.BA", 
                   "CELU.BA", "CGPA2.BA", "CTIO.BA", "DGCU2.BA", "FERR.BA", "FIPL.BA", "GAMI.BA", "GCDI.BA", "GCLA.BA", 
                   "HARG.BA", "HAVA.BA", "INVJ.BA", "IRSA.BA", "LEDE.BA", "LONG.BA", "METR.BA", "MOLA.BA", "MOLI.BA", 
                   "MORI.BA", "OEST.BA", "PATA.BA", "RICH.BA", "RIGO.BA", "SAMI.BA", "SEMI.BA"]

    cedears = ["AAL", "AAPL", "ABBV", "ABEV", "ABNB", "ABT", "ADBE", "ADGO", "ADI", "ADP", "AEM", "AIG", "AMAT", "AMD", 
               "AMGN", "AMZN", "AOCA", "ARCO", "ARKK", "ASR", "AUY", "AVGO", "AXP", "AZN", "BA", "BA.C", "BABA", "BB", 
               "BBD", "BBV", "BCS", "BHP", "BIDU", "BIIB", "BIOX", "BITF", "BK", "BMY", "BNG", "BP", "BRFS", "BRKB", "BSBR", 
               "C", "CAAP", "CAH", "CAR", "CAT", "CBRD", "CDE", "CL", "COIN", "COST", "CRM", "CS", "CSCO", "CVX", "CX", "DD", 
               "DE", "DESP", "DIA", "DISN", "DOCU", "DOW", "E", "EA", "EBAY", "EEM", "EFX", "ERIC", "ERJ", "ETSY", "EWZ", "F", 
               "FCX", "FDX", "FMX", "FSLR", "GE", "GFI", "GGB", "GILD", "GLOB", "GLW", "GM", "GOLD", "GOOGL", "GPRK", "GRMN", 
               "GS", "HAL", "HD", "HL", "HMC", "HMY", "HOG", "HON", "HPQ", "HSBC", "HSY", "HUT", "HWM", "IBM", "IFF", "INTC", 
               "ITUB", "IWM", "JD", "JMIA", "JNJ", "JPM", "KMB", "KO", "KOFM", "LLY", "LMT", "LRCX", "LVS", "LYG", "MA", "MCD", 
               "MDT", "MELI", "META", "MMM", "MO", "MOS", "MRK", "MSFT", "MSI", "MSTR", "MU", "NEM", "NFLX", "NGG", "NIO", "NKE", 
               "NOKA", "NTCO", "NTES", "NUE", "NVDA", "NVS", "ORAN", "ORCL", "OXY", "PAAS", "PAC", "PANW", "PBI", "PBR", "PCAR", 
               "PEP", "PFE", "PG", "PHG", "PKS", "PSX", "PYPL", "QCOM", "QQQ", "RBLX", "RIO", "RTX", "SAN", "SAP", "SATL", "SBUX", 
               "SCCO", "SE", "SHEL", "SHOP", "SI", "SID", "SLB", "SNAP", "SNOW", "SONY", "SPGI", "SPOT", "SPY", "SQ", "SYY", "T", 
               "TEFO", "TEN", "TGT", "TM", "TMO", "TRIP", "TRVV", "TSLA", "TSM", "TTE", "TV", "TWLO", "TXN", "TXR", "UAL", "UBER", 
               "UGP", "UL", "UNH", "UNP", "UPST", "USB", "V", "VALE", "VIST", "VIV", "VOD", "VZ", "WBA", "WFC", "WMT", "X", "XLE", 
               "XLF", "XOM", "XP", "YY", "ZM"]

    adrs = ["BBAR", "BMA", "CEPU", "CRESY", "EDN", "GGAL", "IRS", "LOMA", "PAM", "SUPV", "TEO", "TGS", "TS", "TX", "YPF"]

    sectors = ["XLC", "XLP", "XLY", "XLF", "XLV", "XLI", "XLRE", "XLU", "XBI", "XLB", "XLK", "XLE"]
    
    precarga = ["lideres", "general", "cedears", "adrs", "sectores"]
    precarga_dict = {"lideres" : lideres_arg, "general" : general_arg, "cedears" : cedears, "adrs" : adrs, "sectores" : sectors}
    
    if activos in precarga:
        activos = precarga_dict[activos]
    
    if tipo == "no end":
        data = yf.download(activos, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(activos, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(activos, interval = interval, period= period, progress = False, auto_adjust = True)
    
    if swap:
        #data = data.swaplevel(i = 0, j = 1, axis = 1)
        # Algoritmo para procesar el MultipleTicker download de yfinance
        dicto = {}
        low = data["Low"]
        high = data["High"]
        close = data["Close"]
        open = data["Open"]
        volume = data["Volume"]

        tickers = list(data["Close"].columns)

        for ticker in tickers:
            dicto[ticker] = {
                "Open" : open[ticker],
                "High" : high[ticker],
                "Low" : low[ticker],
                "Close" : close[ticker],
                "Volume" : volume[ticker]
            }

            dicto[ticker] = pd.DataFrame(dicto[ticker])
        return dicto
    return data

In [10]:
# FUNCIONALIDAD PRINCIPAL DEL MÓDULO:

def filterSCSV(activos, timeframe, report = False):
    """
    Esta función utiliza el indicador técnico RSI para determinar el momentum de los activos seleccionados: Sobrecompra (>= 70), medio 
    (menor a 70 y mayor a 30), o bien Sobreventa (<= 30). Adicionalmente se informa la zona de tendencia del activo (muchas veces utilizada 
    como filtro para sistemas de cruces de medias móviles). Si RSI >= 50: Zona Bull, sino, Zona Bear. Devuelve tablas, tablas estiladas e 
    incluso la info en un diccionario clasificando activos por nivel de SC, SV y zona de tendencia.
                       
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en Argentina (puede demorar bastante).
        - timeframe: es la duración de cada sesión/vela (compresión temporal). Puede ser "diario", "semanal" o "mensual".
        - report: si se pasa "True" como argumento, genera un archivo excel en el directorio del script con la tabla estilada.
        
    # Outputs:
        - tabla_final_activos_en_filas: es un DF resumen con los activos en las filas y las variables en las columnas.
        - tabla_final_activos_en_columnas: es un DF resumen con los activos en las columnas y las variables en las filas.
        - estilado: tabla_final_activos_en_filas estilada con colores rojos y verdes según tendencia y un gradient en el ADX.
        - screener: devuelve un diccionario con los listados de activos que están en los distintos rangos de tendencia (clasificación)
    """
    
    import pandas as pd
    import pandas_ta as ta
    import yfinance as yf
    
    if timeframe == "diario":
        interval = "1d"
        period = "1y"
    elif timeframe == "semanal":
        interval = "1wk"
        period = "2y"
    elif timeframe == "mensual":
        interval = "1mo"
        period = "4y"
    else:
        return "timeframe incorrecto, elegir 'diario', 'semanal' o 'mensual'"
    
    data = getDataYfMulti(activos, tipo = "period", interval = interval, period = period, swap = True)
    
    # Defino datos de estrategia  # Features: RSI_14
    MyStrategy = ta.Strategy(
    name = "RSI",
    ta = [
        {"kind" : "rsi"},
        ]
    )

  
    dict_resumenPorActivo = {}  # ==> Resumen por activo (un diccionario de diccionarios).

    i = 0
    
    tickers = list(data.keys())
    
    for activo in tickers:
        try:
            datat = pd.DataFrame(data[activo])
            datat.ta.strategy(MyStrategy)
            datat.dropna(inplace = True)
            datat["estadoRSI"] = np.where(datat["RSI_14"] <= 30, "Sobreventa", 
                                          np.where(((datat["RSI_14"] > 30) & (datat["RSI_14"] < 70)), "medio",
                                                np.where((datat["RSI_14"] >= 70), "Sobrecompra","")))
            
            datat["sentido"] = np.where(datat["RSI_14"] >= 50, "Zona Bull", "Zona Bear")
            
                                          
            
            # Cargamos el activo:
            dict_resumenPorActivo[activo] = {"estadoRSI" : datat["estadoRSI"][-1],
                                             "sentido" : datat["sentido"][-1],
                                             "RSI" : round(datat["RSI_14"][-1],2),
                                             }
            i += 1
            print(f"[{activo}]Procesando {i} de {len(tickers)} activos ...")

        except:
            pass
    
    tabla_final_activos_en_columnas = pd.DataFrame(dict_resumenPorActivo)
    tabla_final_activos_en_filas = tabla_final_activos_en_columnas.transpose()
    
    tabla = tabla_final_activos_en_filas
    
    def set_styles(tabla):
        def highlight_sentido(val, color_if_true, color_if_false):
            color = color_if_true if val == "Zona Bull" else color_if_false
            return f"background-color: {color}; color: black"
                

        styler = tabla.style.applymap(highlight_sentido, color_if_true='#90EF90', color_if_false='#FA6B84', 
                              subset=['sentido']) \
                            .applymap(lambda _: "font-weight: bold") \
                            .background_gradient(subset = ["RSI"], cmap = "Blues") \
                            .format('{:.2f}', subset=["RSI"])
        
        return styler
    
    estilado = set_styles(tabla)
    
    # Generación de reporte en excel (opcional)
    if report == True:
        from datetime import date
        today = date.today()
        estilado.to_excel(f'reporteRSI-{today}.xlsx')
    
    
    sobrecompra = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["estadoRSI"] == "Sobrecompra"].index)
    sobreventa = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["estadoRSI"] == "Sobreventa"].index)
    medio = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["estadoRSI"] == "medio"].index)
    bull = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["sentido"] == "Zona Bull"].index)
    bear = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["sentido"] == "Zona Bear"].index)
    
    screener = {"sobrecomprados" : sobrecompra, "sobrevendidos" : sobreventa,
               "medios" : medio, "zonaBull" : bull, "zonaBear" : bear}
    
    return (tabla_final_activos_en_filas, 
            tabla_final_activos_en_columnas, 
            estilado, 
            screener)

In [11]:
tabla, _, estilada, screener = filterSCSV("adrs", timeframe = "diario", report = False)

[BBAR]Procesando 1 de 15 activos ...
[BMA]Procesando 2 de 15 activos ...
[CEPU]Procesando 3 de 15 activos ...
[CRESY]Procesando 4 de 15 activos ...
[EDN]Procesando 5 de 15 activos ...
[GGAL]Procesando 6 de 15 activos ...
[IRS]Procesando 7 de 15 activos ...
[LOMA]Procesando 8 de 15 activos ...
[PAM]Procesando 9 de 15 activos ...
[SUPV]Procesando 10 de 15 activos ...
[TEO]Procesando 11 de 15 activos ...
[TGS]Procesando 12 de 15 activos ...
[TS]Procesando 13 de 15 activos ...
[TX]Procesando 14 de 15 activos ...
[YPF]Procesando 15 de 15 activos ...


In [12]:
# filterSCSV output - tabla
tabla

,estadoRSI,sentido,RSI
BBAR,medio,Zona Bull,61.0
BMA,medio,Zona Bull,68.76
CEPU,medio,Zona Bull,65.39
CRESY,medio,Zona Bull,56.09
EDN,medio,Zona Bull,61.65
GGAL,medio,Zona Bull,63.71
IRS,medio,Zona Bull,57.66
LOMA,medio,Zona Bull,68.27
PAM,medio,Zona Bull,54.49
SUPV,medio,Zona Bull,63.6


In [13]:
# filterSCSV output - tabla con estilos
estilada

,estadoRSI,sentido,RSI
BBAR,medio,Zona Bull,61.00
BMA,medio,Zona Bull,68.76
CEPU,medio,Zona Bull,65.39
CRESY,medio,Zona Bull,56.09
EDN,medio,Zona Bull,61.65
GGAL,medio,Zona Bull,63.71
IRS,medio,Zona Bull,57.66
LOMA,medio,Zona Bull,68.27
PAM,medio,Zona Bull,54.49
SUPV,medio,Zona Bull,63.60


In [14]:
# filterSCSV output - screener
screener

{'sobrecomprados': [],
 'sobrevendidos': [],
 'medios': ['BBAR',
  'BMA',
  'CEPU',
  'CRESY',
  'EDN',
  'GGAL',
  'IRS',
  'LOMA',
  'PAM',
  'SUPV',
  'TEO',
  'TGS',
  'TS',
  'TX',
  'YPF'],
 'zonaBull': ['BBAR',
  'BMA',
  'CEPU',
  'CRESY',
  'EDN',
  'GGAL',
  'IRS',
  'LOMA',
  'PAM',
  'SUPV',
  'TEO',
  'TGS'],
 'zonaBear': ['TS', 'TX', 'YPF']}